In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ESRGAN (new arch.) on Colab
Official Github Repo: https://github.com/xinntao/ESRGAN <br>
This notebook was curated by M. Ahabb (Ahabbscience Studio)

In [ ]:
!nvidia-smi #recommended gpus are p100 and T4

Sun Jun 18 10:13:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Clone ESRGAN repo

In [ ]:
!git clone https://github.com/xinntao/ESRGAN


Cloning into 'ESRGAN'...
remote: Enumerating objects: 225, done.
remote: Total 225 (delta 0), reused 0 (delta 0), pack-reused 225
Receiving objects: 100% (225/225), 24.86 MiB | 26.99 MiB/s, done.
Resolving deltas: 100% (86/86), done.


This is optional if you want to link your google drive to the notebook to add files or pretrained models of your choice from your google drive.

Run this cell to upload images

In [ ]:
%cd /content/ESRGAN/LR

from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

/content/ESRGAN/LR


Saving data1_5.bmp to data1_5.bmp
Saving data1_9.bmp to data1_9.bmp
Saving data1_43.bmp to data1_43.bmp
Saving data1_45.bmp to data1_45.bmp
Saving data1_48.bmp to data1_48.bmp
Saving data1_49.bmp to data1_49.bmp
Saving data1_54.bmp to data1_54.bmp
Saving data1_55.bmp to data1_55.bmp
Saving data1_58.bmp to data1_58.bmp
Saving data1_59.bmp to data1_59.bmp
Saving data1_62.bmp to data1_62.bmp
Saving data1_63.bmp to data1_63.bmp
Saving data1_64.bmp to data1_64.bmp
Saving data1_68.bmp to data1_68.bmp
Saving data1_71.bmp to data1_71.bmp
Saving data1_72.bmp to data1_72.bmp
Saving data1_74.bmp to data1_74.bmp
Saving data1_75.bmp to data1_75.bmp
Saving data1_76.bmp to data1_76.bmp
Saving data1_81.bmp to data1_81.bmp
Saving data1_87.bmp to data1_87.bmp
Saving data1_88.bmp to data1_88.bmp
Saving data1_90.bmp to data1_90.bmp
Saving data1_95.bmp to data1_95.bmp
Saving data2_0.bmp to data2_0.bmp
Saving data2_13.bmp to data2_13.bmp
Saving data2_15.bmp to data2_15.bmp
Saving data2_18.bmp to data2_18.bm

Run this cell if you want to upload your video. Make sure your video filename contains no spaces, e.g, "my_video" - not "my video". And upload one video per operation.

Download pretrained models

In [ ]:
import gdown
print("Downloading pretrained models")
output1 = '/content/ESRGAN/models/RRDB_ESRGAN_x4.pth'
output2 = '/content/ESRGAN/models/RRDB_PSNR_x4.pth'
output3 = '/content/ESRGAN/models/PPON_D.pth'
output4 = '/content/ESRGAN/models/PPON_G.pth'
print ('Downloading RRDB_ESRGAN_x4.pth')
gdown.download('https://drive.google.com/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene', output1, quiet=True)
print ('Downloading RRDB_PSNR_x4.pth')
gdown.download('https://drive.google.com/uc?id=1pJ_T-V1dpb1ewoEra1TGSWl5e6H7M4NN', output2, quiet=True)
print ('Downloading PPON_D.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=1Fr5aKCD6mw6P-hI0BZr6My2gHNhtUk-V', output3, quiet=True)
print ('Downloading PPON_G.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=12uR3BSftNA0HDYiKda23GyAj_crpSjOm', output4, quiet=True)

'/content/ESRGAN/models/PPON_G.pth'

In [ ]:
#@title Upscale images/video frames
%cd /content/ESRGAN

import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import requests
import imageio
import requests
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

Choose_device = "cuda"  #@param ["cuda","cpu"]

model_path = 'models/RRDB_ESRGAN_x4.pth' #@param ['models/RRDB_ESRGAN_x4.pth','models/RRDB_PSNR_x4.pth','models/PPON_G.pth','models/PPON_D.pth']
device = torch.device(Choose_device)


test_img_folder = 'LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}.png'.format(base), output)

/content/ESRGAN
Model path models/RRDB_ESRGAN_x4.pth. 
Testing...
1 data7_356
2 data4_332
3 data20_339
4 data21_134
5 data2_460
6 data22_230
7 data20_396
8 data4_374
9 data6_252
10 data9_111
11 data22_490
12 data5_2444
13 data4_182
14 data9_368
15 data21_406
16 data21_138
17 data9_302
18 data2_521
19 data8_75
20 data21_3
21 data2_0
22 data7_35
23 data21_472
24 data6_389
25 data6_25
26 data9_344
27 data3_10
28 data7_256
29 data4_115
30 data2_566
31 data20_250
32 data6_181
33 data8_9
34 data7_254
35 data3_37
36 data21_16
37 data9_98
38 data22_71
39 data2_80
40 data2_392
41 data7_228
42 data7_141
43 data8_328
44 data5_506
45 data20_305
46 data20_299
47 data5_2200
48 data22_391
49 data8_255
50 data2_234
51 data2_275
52 data9_89
53 data22_38
54 data6_298
55 data9_218
56 data6_359
57 data9_296
58 data20_203
59 data8_31
60 data3_89
61 data21_93
62 data20_289
63 data2_53
64 data2_426
65 data3_6
66 data9_70
67 data22_402
68 data7_290
69 data9_320
70 data1_62
71 data6_274
72 data20_3
73 data20_3

Run this cell to encode your results into a video.

In [ ]:
!ffmpeg -f image2 -framerate 25 -i /content/ESRGAN/results/%04d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p output.mp4

Make a zip file of your results

In [ ]:
from google.colab import files

!zip -r results.zip /content/ESRGAN/results

  adding: content/ESRGAN/results/ (stored 0%)
  adding: content/ESRGAN/results/data4_184.png (deflated 2%)
  adding: content/ESRGAN/results/data7_52.png (deflated 1%)
  adding: content/ESRGAN/results/data2_376.png (deflated 1%)
  adding: content/ESRGAN/results/data7_331.png (deflated 1%)
  adding: content/ESRGAN/results/data22_356.png (deflated 0%)
  adding: content/ESRGAN/results/data4_262.png (deflated 2%)
  adding: content/ESRGAN/results/data4_306.png (deflated 2%)
  adding: content/ESRGAN/results/data8_194.png (deflated 1%)
  adding: content/ESRGAN/results/data3_16.png (deflated 1%)
  adding: content/ESRGAN/results/data20_219.png (deflated 1%)
  adding: content/ESRGAN/results/data21_342.png (deflated 0%)
  adding: content/ESRGAN/results/data9_353.png (deflated 1%)
  adding: content/ESRGAN/results/data3_1.png (deflated 1%)
  adding: content/ESRGAN/results/data1_74.png (deflated 1%)
  adding: content/ESRGAN/results/data5_860.png (deflated 1%)
  adding: content/ESRGAN/results/data20_3

Download Zip file

In [ ]:
files.download('results.zip')